In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
### Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
### Preprocess the data
# Drop unnecessary columns
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
### Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### One-hot encode the 'Geography' column
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
geography_encoded = ohe.fit_transform(data[['Geography']]).toarray()
geography_encoded_df = pd.DataFrame(geography_encoded, columns=ohe.get_feature_names_out(['Geography']))

data = pd.concat([data, geography_encoded_df], axis=1)
data.drop(['Geography'], axis=1, inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
### Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('ohe.pkl', 'wb') as file:
    pickle.dump(ohe, file)


In [8]:
### Devide the data into dependent and independent variables
X = data.drop(['Exited'], axis=1)
y = data['Exited']

In [9]:
### Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
### Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
### Save the scaler 
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Implementation

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#### Build the ANN Model

In [14]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ## HL1 which is connected to input layer
        Dense(32, activation='relu'),   ## HL2 which is connected to HL1
        Dense(1, activation='sigmoid')  ## Output layer which is connected to HL2
    ]
)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [17]:
### Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
### Set up TensorBoard logging
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [26]:
### Setup early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

### restore_best_weights=True --> When we are going through the forward and backward propagation, 
### then at which epoch the loss is minimum, that weights will be restored.

In [21]:
### Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100


250/250 [==============================] - 4s 7ms/step - loss: 0.3935 - accuracy: 0.8369 - val_loss: 0.3420 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3550 - accuracy: 0.8565 - val_loss: 0.3453 - val_accuracy: 0.8595
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3490 - accuracy: 0.8558 - val_loss: 0.3416 - val_accuracy: 0.8565
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3419 - accuracy: 0.8595 - val_loss: 0.3885 - val_accuracy: 0.8545
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3417 - accuracy: 0.8625 - val_loss: 0.3513 - val_accuracy: 0.8525
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3349 - accuracy: 0.8640 - val_loss: 0.3376 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3374 - accuracy: 0.8609 - val_loss: 0.3375 - val_accuracy: 0.86

In [22]:
model.save('model.h5')

c:\Users\adity\Desktop\Udemy ML DL\dl-venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
### Load the tensorboard Extension
%load_ext tensorboard

In [25]:
### Load the tensorboard logs
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 20544), started 0:01:57 ago. (Use '!kill 20544' to kill it.)

#### Prediction

In [27]:
### Load the pickle files
from tensorflow.keras.models import load_model

model = load_model('model.h5')

### Load the encoders and scaler
with open('ohe.pkl', 'rb') as file:
    ohe = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

In [50]:
### Example input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [51]:
### Converting the categorical variables into numerical variables
geo_encoded = ohe.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out(['Geography']))

geo_encoded_df

c:\Users\adity\Desktop\Udemy ML DL\dl-venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [52]:
input_df = pd.DataFrame([input_data])

In [53]:
### Encode the categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [54]:
### Concatenate the encoded geography with the input data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [58]:
### Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [60]:
### Predict the churn probability
prediction = model.predict(input_scaled)

1/1 [==============================] - 0s 16ms/step


In [62]:
prediction

array([[0.04226297]], dtype=float32)

In [63]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
